In [15]:
from uuid import uuid4
from datetime import datetime
import json

class BaseModel:
    def __init__(self, *args, **kwargs):
        """Initialize the base model"""
        from models import storage  # to avoid circular import
        if not kwargs:  # if no dictionary (key&value) argument is passed
            self.id = str(uuid4())
            self.created_at = datetime.now()
            self.updated_at = datetime.now()
            storage.new(self)
        else:
            for k, v in kwargs.items():
                if k != "__class__":
                    if k in ("created_at", "updated_at"):
                        # Construct a date from the output of date.isoformat()
                        # setattr(x, 'y', v) is equivalent to x.y = v''
                        setattr(self, k, datetime.fromisoformat(v))
                    # otherwise create a key and value pair of other attributes
                    else:
                        setattr(self, k, v)

    def __str__(self):
        """Returns a customized string representation

        [<class name>] (<self.id>) <self.__dict__>
        """
        return f"[{type(self).__name__}] ({self.id}) {self.__dict__}"

    def save(self):
        from models import storage  # to avoid circular import
        objects = []
        self.updated_at = datetime.now()
        storage.save()

    def to_dict(self):
        dict_1 = self.__dict__.copy()  # create a copy of __dict__
        # add __class__ key with class name as value
        dict_1["__class__"] = self.__class__.__name__
        for k, v in self.__dict__.items():
            if k in ("created_at", "updated_at"):
                # self.__dict__() returns a dict of
                # instance attributes and values
                # print("Here==>",self.__dict__)
                # [k] simply sends in a key and gets it's
                # value in our case, created_at or updated_at, then;
                # converts datetime{2022-11-21 16:06:40.075755}
                # to isoformat {2022-11-21T16:06:40.075755}
                v = self.__dict__[k].isoformat()
                dict_1[k] = v
        return dict_1


In [32]:
class_dict = {"BaseModel": BaseModel}


class FileStorage():
    # private class attributes
    __file_path = "file.json"
    __objects = {}

    def all(self):
        return self.__objects

    def new(self, obj):
        self.__objects["{}.{}".format(obj.__class__.__name__, obj.id)] = obj

    def save(self):
        with open(self.__file_path, mode="w") as f:
            dict_storage = {}
            for k, v in self.__objects.items():
                dict_storage[k] = v.to_dict()
            json.dump(dict_storage, f)

    def reload(self):
        try:
            with open(self.__file_path, 'r') as f:
                # load the json file as instance object
                instance = json.load(f)
                for key in instance:
                    self.__objects[key] = (
                        class_dict[instance[key]["__class__"]](**instance[key])
                        )
        except Exception:
            pass
    

storage = FileStorage()
storage.reload()

In [30]:
__objects = {}
def reload():
    try:
        with open("file.json", 'r') as f:
            # load the json file as instance object
            instance = json.load(f)
            for key in instance:
                __objects[key] = (class_dict[instance[key]["__class__"]](**instance[key]))
            print(__objects[key])
    except:
        pass

In [31]:
reload()

[BaseModel] (3a661e3f-c828-4378-ae22-696a1d566915) {'id': '3a661e3f-c828-4378-ae22-696a1d566915', 'created_at': datetime.datetime(2022, 11, 24, 14, 20, 32, 233837), 'updated_at': datetime.datetime(2022, 11, 24, 14, 20, 32, 233843), 'name': 'My_First_Model', 'my_number': 89}


In [19]:
my_model = BaseModel()
my_model.name = "My_First_Model"
my_model.my_number = 89
my_model.save()

AttributeError: 'BaseModel' object has no attribute 'to_dict'

In [8]:
my_model.all()

AttributeError: 'BaseModel' object has no attribute 'all'

In [1]:
loaded_dict = {"BaseModel.31cd9a0b-2adc-43ce-97e6-f44bbe10c7d7": {"id": "31cd9a0b-2adc-43ce-97e6-f44bbe10c7d7", "created_at": "2022-11-24T14:40:47.123434", "updated_at": "2022-11-24T14:40:47.123458", "name": "My_First_Model", "my_number": 89, "__class__": "BaseModel"}, "BaseModel.59b2d43d-0951-4b6d-88ed-3771487c1732": {"id": "59b2d43d-0951-4b6d-88ed-3771487c1732", "created_at": "2022-11-24T14:51:30.037868", "updated_at": "2022-11-24T14:51:30.037879", "name": "My_First_Model", "my_number": 89, "__class__": "BaseModel"}, "BaseModel.ac40d95e-31a6-40fc-9345-e891994938e0": {"id": "ac40d95e-31a6-40fc-9345-e891994938e0", "created_at": "2022-11-24T14:52:07.429135", "updated_at": "2022-11-24T14:52:07.429150", "name": "My_First_Model", "my_number": 89, "__class__": "BaseModel"}}

In [4]:
objects = {}
from models.base_model import BaseModel
class_dict = {"BaseModel": BaseModel}
for key in loaded_dict.keys():
    # use each loaded_dict key to create a k:v pair in __objects
    # values are
    objects[key] = (
        class_dict[loaded_dict[key]["__class__"]](**loaded_dict[key])
        )
    print(class_dict[loaded_dict[key]["__class__"]](**loaded_dict[key]),"\n")

[BaseModel] (31cd9a0b-2adc-43ce-97e6-f44bbe10c7d7) {'id': '31cd9a0b-2adc-43ce-97e6-f44bbe10c7d7', 'created_at': datetime.datetime(2022, 11, 24, 14, 40, 47, 123434), 'updated_at': datetime.datetime(2022, 11, 24, 14, 40, 47, 123458), 'name': 'My_First_Model', 'my_number': 89} 

[BaseModel] (59b2d43d-0951-4b6d-88ed-3771487c1732) {'id': '59b2d43d-0951-4b6d-88ed-3771487c1732', 'created_at': datetime.datetime(2022, 11, 24, 14, 51, 30, 37868), 'updated_at': datetime.datetime(2022, 11, 24, 14, 51, 30, 37879), 'name': 'My_First_Model', 'my_number': 89} 

[BaseModel] (ac40d95e-31a6-40fc-9345-e891994938e0) {'id': 'ac40d95e-31a6-40fc-9345-e891994938e0', 'created_at': datetime.datetime(2022, 11, 24, 14, 52, 7, 429135), 'updated_at': datetime.datetime(2022, 11, 24, 14, 52, 7, 429150), 'name': 'My_First_Model', 'my_number': 89} 



In [9]:
abc = **loaded_dict

TypeError: eval() takes no keyword arguments

In [17]:

def function_creator():
 
    # expression to be evaluated
    expr = input("Enter the function(in terms of x):")
 
    # variable used in expression
    x = int(input("Enter the value of x:"))
 
    # evaluating expression
    y = eval(expr)
    
    print(expr)
    # printing evaluated result
    print("y =", y)

In [18]:
function_creator()

(x**2)+(3*x)+2
y = 42


In [21]:
eval("function_creator()")

2+x
y = 7


In [23]:
vals = {'id': 'a563a2f2-3d00-4a73-87bd-a692827b81f0', 'created_at': '2022-11-24T15:27:43.727633', 'updated_at': '2022-11-24T15:27:43.727640', 'name': 'My_First_Model', 'my_number': 89, '__class__': 'BaseModel'}

def test_class_kwargs(**kwargs):
    for k,v in kwargs.items():
        print(f"{k}: {v}")

In [27]:
new = eval("BaseModel")(**vals)

In [30]:

print(new)

[BaseModel] (a563a2f2-3d00-4a73-87bd-a692827b81f0) {'id': 'a563a2f2-3d00-4a73-87bd-a692827b81f0', 'created_at': datetime.datetime(2022, 11, 24, 15, 27, 43, 727633), 'updated_at': datetime.datetime(2022, 11, 24, 15, 27, 43, 727640), 'name': 'My_First_Model', 'my_number': 89}


In [33]:
new.__class__.__name__

'BaseModel'

In [11]:
def do_create(*args):
    if len(args) > 1:
        new_instance = f"{args[0]} ==> {args[1:]}"
        print(new_instance)

In [12]:
do_create("hello","world")

hello ==> ('world',)


In [1]:
vals = {'id': 'a563a2f2-3d00-4a73-87bd-a692827b81f0', 'created_at': '2022-11-24T15:27:43.727633', 'updated_at': '2022-11-24T15:27:43.727640', 'name': 'My_First_Model', 'my_number': 89, '__class__': 'BaseModel'}
del vals['id']

In [2]:
vals

{'created_at': '2022-11-24T15:27:43.727633',
 'updated_at': '2022-11-24T15:27:43.727640',
 'name': 'My_First_Model',
 'my_number': 89,
 '__class__': 'BaseModel'}

In [25]:
bases = {
	"GazeModel.18198c63-4fa1-4563-88d5-a161e94e71f3": {
		"id": "18198c63-4fa1-4563-88d5-a161e94e71f3",
		"created_at": "2022-11-25T12:34:51.621151",
		"updated_at": "2022-11-25T12:34:51.621168",
		"name": "My_First_Model",
		"my_number": 89,
		"__class__": "BaseModel"
	},
	"DaseModel.398435d2-5736-48b5-8ea9-9ad5a8d930aa": {
		"id": "398435d2-5736-48b5-8ea9-9ad5a8d930aa",
		"created_at": "2022-11-25T20:15:06.184575",
		"updated_at": "2022-11-25T20:15:06.184620",
		"__class__": "BaseModel"
	},
	"BaseModel.8346bddd-183c-4e82-ac9a-6ba5ca55c302": {
		"id": "8346bddd-183c-4e82-ac9a-6ba5ca55c302",
		"created_at": "2022-11-25T20:15:30.529033",
		"updated_at": "2022-11-25T20:15:30.529081",
		"__class__": "BaseModel"
	}
}


In [19]:
if "GazeModel" in list(bases.keys()).split("."):
    print(bases)

AttributeError: 'list' object has no attribute 'split'

In [23]:
import re
regex = 'BaseModel'
 
match = re.findall(regex, my_list)
print(match)

TypeError: expected string or bytes-like object

In [9]:
my_list = ["[FaseModel] (18198c63-4fa1-4563-88d5-a161e94e71f3) {'id': '18198c63-4fa1-4563-88d5-a161e94e71f3', 'created_at': datetime.datetime(2022, 11, 25, 12, 34, 51, 621151), 'updated_at': datetime.datetime(2022, 11, 25, 12, 34, 51, 621168), 'name': 'My_First_Model', 'my_number': 89}", "[BaseModel] (398435d2-5736-48b5-8ea9-9ad5a8d930aa) {'id': '398435d2-5736-48b5-8ea9-9ad5a8d930aa', 'created_at': datetime.datetime(2022, 11, 25, 20, 15, 6, 184575), 'updated_at': datetime.datetime(2022, 11, 25, 20, 15, 6, 184620)}", "[BaseModel] (8346bddd-183c-4e82-ac9a-6ba5ca55c302) {'id': '8346bddd-183c-4e82-ac9a-6ba5ca55c302', 'created_at': datetime.datetime(2022, 11, 25, 20, 15, 30, 529033), 'updated_at': datetime.datetime(2022, 11, 25, 20, 15, 30, 529081)}"]

In [15]:
for i in len(my_list)
if my_list[0].split()[0].strip("[]")

'FaseModel'

In [28]:
for keys in bases.keys():
    if str(keys).startswith("Base"):
        print(keys)

BaseModel.8346bddd-183c-4e82-ac9a-6ba5ca55c302


In [56]:
lst = """User.update("38f22813-2753-4d42-b37c-57a17f1e4f88", {'first_name': "John", "age": 89})"""
lst = lst.replace("(", " ").replace(".", " ")\
            .replace(")", "").replace(",", " ").replace(":", "")\
                .replace("{", " ").replace("}", " ").replace("  ", " ")

In [58]:
import shlex

lst = shlex.split(lst)
lst

['User',
 'update',
 '38f22813-2753-4d42-b37c-57a17f1e4f88',
 'first_name',
 'John',
 'age',
 '89']

In [53]:

to_dct = lst[3:]
dic = {to_dct[i]: to_dct[i + 1] for i in range(0, len(to_dct), 2)}

In [54]:
for k,v in dic.items():
    

{'first_name': 'John', 'age': '89'}

In [51]:
to_dct

['first_name', 'John', 'age', '89']

In [1]:
class_dict = {
    "BaseModel": "BaseModel",
    "User": "User",
    "Amenity": "Amenity",
    "City": "City",
    "Place": "Place",
    "Review": "Review"
    }

In [60]:
lists = [i for i in class_dict.keys()]

In [61]:
user_dict = {
    "name": "BaseModel",
    "User_id": "wer8e74u",
    }

['BaseModel', 'User', 'Amenity', 'City', 'Place', 'Review']

In [2]:
func(name="BaseModel", User_id="wer8e74u")
func(**user_dict)

TypeError: print() takes at most 4 keyword arguments (6 given)

In [ ]:
lst = ['BaseModel', 'User', 'Amenity', 'City', 'Place', 'Review']